In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2, 
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('images/train',
                                                target_size = (64,64),
                                                batch_size = 64,
                                                class_mode = 'binary')

test_set = test_datagen.flow_from_directory('images/test',
                                           target_size = (64,64),
                                           batch_size = 64,
                                           class_mode = 'binary')

In [3]:
X_train, X_test = train_test_split(training_set, test_size = 0.2)

In [4]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution 1
classifier.add(Convolution2D(32,(3,3), input_shape = (64,64,3), activation = 'relu'))
classifier.add(Convolution2D(32,(3,3), activation = 'relu'))
classifier.add(Dropout(0.2))

# Step 2 - Pooling 1
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 3 - Convolution 2
classifier.add(Convolution2D(64,(3,3), padding='same', activation = 'relu'))
classifier.add(Convolution2D(64,(3,3), padding='same', activation = 'relu'))
classifier.add(Dropout(0.2))

# Step 4 - Pooling 2
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 5 - Convolution 3
classifier.add(Convolution2D(128,(3,3), padding='same', activation = 'relu'))
classifier.add(Convolution2D(128,(3,3), padding='same', activation = 'relu'))
classifier.add(Dropout(0.2))

# Step 6 - Pooling 3
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 3 - Flattening
classifier.add(Flatten())
#Step 4 - Full Connection
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(1, activation = 'sigmoid'))

# Compiling the CNN
# opt = SGD(learning_rate=0.001, momentum=0.9)
opt = Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name="Adam",)
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = classifier.fit(training_set,
                                  steps_per_epoch = 75,
                                  epochs = 200,
                                  validation_data = test_set)

Epoch 1/200
75/75 [==============================] - 54s 726ms/step - loss: 0.0787 - accuracy: 0.9900 - val_loss: 0.2172 - val_accuracy: 0.9421
Epoch 2/200
75/75 [==============================] - 49s 658ms/step - loss: 0.0810 - accuracy: 0.9877 - val_loss: 0.2209 - val_accuracy: 0.9421
Epoch 3/200
75/75 [==============================] - 44s 581ms/step - loss: 0.5616 - accuracy: 0.9875 - val_loss: 0.2422 - val_accuracy: 0.9421
Epoch 4/200
75/75 [==============================] - 47s 634ms/step - loss: 0.1546 - accuracy: 0.9869 - val_loss: 0.2646 - val_accuracy: 0.9421
Epoch 5/200
75/75 [==============================] - 46s 618ms/step - loss: 0.0911 - accuracy: 0.9887 - val_loss: 0.2596 - val_accuracy: 0.9421
Epoch 6/200
75/75 [==============================] - 45s 600ms/step - loss: 0.0958 - accuracy: 0.9844 - val_loss: 0.2232 - val_accuracy: 0.9421
Epoch 7/200
75/75 [==============================] - 44s 585ms/step - loss: 0.0831 - accuracy: 0.9859 - val_loss: 0.2218 - val_accuracy:

In [1]:
# plot history
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'test')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [13]:
def image_test(picture_name):
    # Loading an image from the disk
    test_image = image.load_img(picture_name, target_size = (64,64))

    # Converting the image to a numpy array
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    test_image = test_image / 255
    result = classifier.predict(test_image)

    # our cut off
    
    if result[0][0] >= 0.5:
        prediction = 'Dandelion'
    else:
        prediction = 'Not a Dandelion'
    print(prediction)
    print(result[0][0])

In [14]:
image_test('test_image.jpg')

image_test('random_dog.png')
image_test('16643944275_3cd4cd966c.jpg')

Dandelion
0.98754543
Dandelion
0.98754543
Dandelion
0.98754543
Dandelion
0.98754543


In [15]:
image_test('test_image2.jpg')

Dandelion
0.98754543
